In [1]:
!update
!pip install datasets

/bin/bash: line 1: update: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("bnadimi/PyraNet-Verilog")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

PyraNetOnVeriBest.csv:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/692238 [00:00<?, ? examples/s]

In [3]:
train_dataset = ds['train']
print(train_dataset[:1])


{'code': ["module VCC (output V);\n   assign V = 1'b1;\nendmodule"], 'description': ['{"description": "The Verilog code defines a module named `VCC` that outputs a constant high logic level (1). The output `V` is always set to 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}']}


In [4]:
import json
def convert_format(example):
      code_text = example["code"]  # 取 code 的第一個元素
      description_json = example["description"]  # 取 description 的第一個元素
      return {
          "conversations": [
              {"role": "user", "content": code_text.strip()},
              {"role": "assistant", "content": description_json.strip()}
          ]
      }

example = train_dataset[0]
print(example)


# 使用 map 方法對整個資料集進行轉換
dataset_with_conversations = train_dataset.map(convert_format, remove_columns=["code", "description"])


# 顯示前 5 筆資料以檢查結果
print(dataset_with_conversations[0])


{'code': "module VCC (output V);\n   assign V = 1'b1;\nendmodule", 'description': '{"description": "The Verilog code defines a module named `VCC` that outputs a constant high logic level (1). The output `V` is always set to 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}'}


Map:   0%|          | 0/692238 [00:00<?, ? examples/s]

{'conversations': [{'content': "module VCC (output V);\n   assign V = 1'b1;\nendmodule", 'role': 'user'}, {'content': '{"description": "The Verilog code defines a module named `VCC` that outputs a constant high logic level (1). The output `V` is always set to 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}', 'role': 'assistant'}]}


### Installation

In [5]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [11]:
from datasets import load_dataset

# 加載資料集
dataset = load_dataset("json", data_files="/content/drive/MyDrive/dataset.json", split="train")

# 定義轉換函數，將資料轉換為 conversations 格式
def add_conversations(example):
    return {
        "conversations": [
            {"role": "user", "content": example["Instruction"].strip()},
            {"role": "assistant", "content": example["Response"][0].strip()}  # 假設 Response 是列表
        ]
    }

# 執行 map 函數將資料集轉換
dataset = dataset.map(add_conversations, remove_columns=["Instruction", "Response"])
print(dataset[0])



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/26532 [00:00<?, ? examples/s]

{'conversations': [{'content': 'Implement a module for a simple calculator that can perform basic arithmetic operations. The module should have two inputs, `a` and `b`, and four outputs, `add`, `sub`, `mul`, and `div`. The module should be able to perform the following operations:\n- `add`: add `a` and `b` and output the result\n- `sub`: subtract `b` from `a` and output the result\n- `mul`: multiply `a` and `b` and output the result\n- `div`: divide `a` by `b` and output the result\n\nIf `b` is zero, the `div` output should be zero. The module should also have a reset input, `reset_n`, which should reset all outputs to zero when active low.', 'role': 'user'}, {'content': 'module calculator(\n    input logic [31:0] a,\n    input logic [31:0] b,\n    input logic reset_n,\n    output logic [31:0] add,\n    output logic [31:0] sub,\n    output logic [31:0] mul,\n    output logic [31:0] div\n);\n\n    always @(a, b, reset_n) begin\n        if(!reset_n) begin\n            add <= 0;\n        

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [12]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [14]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset_with_conversations = standardize_sharegpt(dataset_with_conversations)
dataset_with_conversations = dataset_with_conversations.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/26532 [00:00<?, ? examples/s]

Map:   0%|          | 0/26532 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/692238 [00:00<?, ? examples/s]

Map:   0%|          | 0/692238 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [15]:
dataset_with_conversations[0]["conversations"]

[{'content': "module VCC (output V);\n   assign V = 1'b1;\nendmodule",
  'role': 'user'},
 {'content': '{"description": "The Verilog code defines a module named `VCC` that outputs a constant high logic level (1). The output `V` is always set to 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}',
  'role': 'assistant'}]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_with_conversations,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/692238 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [18]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/692238 [00:00<?, ? examples/s]

We verify masking is actually done:

In [19]:
tokenizer.decode(trainer.train_dataset[7]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nmodule invert (input wire i, output wire o);\n   assign o =!i;\nendmodule<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{"description": "The Verilog code defines a module called `invert` that takes a single input `i` and produces an output `o` that is the logical negation (inversion) of `i`. If `i` is 1, `o` will be 0, and if `i` is 0, `o` will be 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}<|eot_id|>'

In [21]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                {"description": "The Verilog code defines a half-adder that computes the sum and carry of two single-bit binary inputs, A and B. It produces a sum output (Sum) using the XOR operation and a carry output (Cout) using the AND operation.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [22]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.32 GB of memory reserved.


In [25]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 692,238 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,0.001900
2,-0.000100
3,0.002500
4,0.000400
5,0.010100
6,0.000600
7,-0.000100
8,0.001100
9,-0.000000
10,-0.000000


Step,Training Loss
1,0.001900
2,-0.000100
3,0.002500
4,0.000400
5,0.010100
6,0.000600
7,-0.000100
8,0.001100
9,-0.000000
10,-0.000000


In [26]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1134.522 seconds used for training.
18.91 minutes used for training.
Peak reserved memory = 7.385 GB.
Peak reserved memory for training = 5.065 GB.
Peak reserved memory % of max memory = 50.098 %.
Peak reserved memory for training % of max memory = 34.36 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [27]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Implement a 2 to 1 mux"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nImplement a 2 to 1 mux<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere is an example implementation of a 2 to 1 mux in Python using the Arduino IDE:\n\n```python\nimport Arduino\n\n# Define the inputs and output pins\ninput_pins = [9, 10]  # Pins for inputs A and B\noutput_pin = 11     # Pin for output Q\n\n# Define the mux function\ndef mux(a, b):\n    if a:\n        output_pin = output_pin ^ 1\n    else:\n        output_pin = output_pin ^ 1\n\n    # Toggle the output pin\n    ArduinodigitalWrite(output_pin, ArduinoHIGH)\n\n# Define the main function\ndef main():\n    while True:\n        # Get the input values\n        input_a = ArduinodigitalRead(input_pins[0])\n        input_b = ArduinodigitalRead(input_pins[1])\n\n        # Apply the mux function\n        mux(input_a, input_b)\n\nif 

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [29]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Implement a verilog 2 to 1 mux "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 500,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Here is a simple implementation of a 2-to-1 multiplexer (MUX) in Verilog:

```verilog
module mux2to1(a0, a1, s, output, output_signal);
input [1:0] a0, a1;
input s;
output output;
output output_signal;
output [1:0] output_signal;

output_signal = (s == 0)? a0 : a1;

assign(output_signal = output_signal);
assign(output = output_signal);

endmodule
```

This module takes two inputs `a0` and `a1`, a select input `s`, and outputs `output` and `output_signal`. The `output_signal` is the result of selecting between `a0` and `a1` based on the value of `s`.

You can use this module like this:

```verilog
module test;
input clk;
input clk enable;
output clk enableclk;
outputclk enableclkclk;

clk enableclkclk = clk enableclkclk;

mux2to1(a0, a1, s, output, output_signal);
// assuming a0 = 0, a1 = 1, s = 0
output_signal = 0;
output = output_signal;
assert(output == output_signal);
assert(output_signal == output_signal);
endmodule
```

In this test module, `a0` and `a1` are initialized to `0` and

!!目前只執行到這邊!!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [32]:
model.save_pretrained("/content/drive/MyDrive/Model2025 3 19")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Model2025 3 19")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/Model2025 3 19/tokenizer_config.json',
 '/content/drive/MyDrive/Model2025 3 19/special_tokens_map.json',
 '/content/drive/MyDrive/Model2025 3 19/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

> 加入區塊引述符號



In [34]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Model2025/3/19/my_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "I would like you to implement a verilog module named TopModule with the following interface. All input and output ports are one bit unless otherwise specified. The module should have one output, 'zero'. The module should always outputs a LOW."},]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 300,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2025.3.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Here is a simple Verilog module that matches your requirements:

``` verilog
module TopModule(
    // No inputs
    // No outputs
    // But we still need to declare a signal to represent 'zero'
    logic zero
);
    
    // always statement to always set zero to LOW
    always zero <= 0;
endmodule
```

However, if you want to write a module that always outputs 'zero', you need to declare both the output and the input signals, and also define the module's interface.

``` verilog
module TopModule(
    input logic clock,
    input logic reset,
   

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


In [ ]:
from google.colab import files
import zipfile

# 上傳 zip 檔案
uploaded = files.upload()

# 解壓縮 ZIP 檔案
with zipfile.ZipFile("my_verilog_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/verilog_dataset")



Saving my_verilog_dataset.zip to my_verilog_dataset.zip


In [ ]:
import os

# 定義資料集路徑
dataset_path = "/content/verilog_dataset"

# 讀取某一問題的檔案，這裡用通配符去匹配相應的檔案
def read_files(question_id):
    # 根據問題編號來動態匹配檔案
    prompt_file = None
    ref_file = None
    test_file = None

    # 取得目錄下所有檔案
    files = os.listdir(dataset_path)

    # 根據問題編號和題目名稱來查找相應的檔案
    for file in files:
        if file.startswith(f"Prob{question_id}_") and file.endswith("_prompt.txt"):
            prompt_file = os.path.join(dataset_path, file)
        elif file.startswith(f"Prob{question_id}_") and file.endswith("_ref.sv"):
            ref_file = os.path.join(dataset_path, file)
        elif file.startswith(f"Prob{question_id}_") and file.endswith("_test.sv"):
            test_file = os.path.join(dataset_path, file)

    if prompt_file and ref_file and test_file:
        # 讀取檔案
        with open(prompt_file, 'r') as file:
            prompt = file.read()

        with open(ref_file, 'r') as file:
            ref_code = file.read()

        with open(test_file, 'r') as file:
            test_code = file.read()

        return prompt, ref_code, test_code
    else:
        raise FileNotFoundError(f"檔案未找到：Prob{question_id}_*")

# 讀取第 1 題（假設問題編號是 "001"）
prompt, ref_code, test_code = read_files("007")

# 顯示資料
print("Prompt:", prompt)
print("Reference Verilog Code:", ref_code)
print("Test Verilog Code:", test_code)


Prompt: 
I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in
 - output out

The module should behave like a wire.


Reference Verilog Code: 
module RefModule (
  input in,
  output out
);

  assign out = in;

endmodule


Test Verilog Code: `timescale 1 ps/1 ps
`define OK 12
`define INCORRECT 13


module stimulus_gen (
	input clk,
	output reg in,
	output reg[511:0] wavedrom_title,
	output reg wavedrom_enable
);


// Add two ports to module stimulus_gen:
//    output [511:0] wavedrom_title
//    output reg wavedrom_enable

	task wavedrom_start(input[511:0] title = "");
	endtask
	
	task wavedrom_stop;
		#1;
	endtask	



	initial begin
		wavedrom_start("Output should follow input");
		repeat(20) @(posedge clk, negedge clk)
			in <= $random;
		wavedrom_stop();
		repeat(100) @(posedge clk, negedge clk) begin
			in <= $random;
		end
		
		#1 $finish;
	end
	
endmodule

module tb();


In [ ]:
!apt-get update
!apt-get install -y iverilog

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,731 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,934 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.

In [ ]:
import re
import subprocess
from unsloth.chat_templates import get_chat_template

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/my_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

pass_count = 0
total_tests = 156

def compile_verilog(code):
    with open("temp.v", "w") as f:
        f.write(code)

    result = subprocess.run(["iverilog", "temp.v"], capture_output=True)
    return result.returncode == 0


for i in range(1, 157):
  messages = [
      {"role": "user", "content": read_files(str(i).zfill(3))[0]},
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, temperature = 1.5, min_p = 0.1)
  decoded_output = tokenizer.batch_decode(outputs)[0]
  code_start = decoded_output.find("module TopModule")  # 假設代碼部分從這一行開始
  code_end = decoded_output.find("endmodule") + len("endmodule")  # 假設代碼在endmodule結尾
  # 保留代碼區域
  final_code = decoded_output[code_start:code_end]
  #print(final_code + "\n")
  if compile_verilog(final_code):
    pass_count += 1


print(pass_count)
pass_at_metric = pass_count / total_tests
print(f"pass@ metric: {pass_at_metric * 100:.2f}%")

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
31
pass@ metric: 19.87%
